# Economic Loss Calculation

In [8]:
import pandas as pd

In [9]:
def get_state_name(lat, long):
    from geopy.geocoders import Nominatim
    locator = Nominatim(user_agent="myGeocoder")
    coordinates = str(lat)+", "+str(long)
    location = locator.reverse(coordinates)
    return location.raw['address']["state"]

In [10]:
def get_value_per_acre(lat, long):
    propertydata = pd.read_csv("Data/statewise-pricing-usa.csv")
    state = get_state_name(lat, long)
    return propertydata.loc[propertydata['State'] == state].values[0,1]

In [11]:
def get_total_economic_loss(lat, long, area):
    acres = 247.105 * area
    return get_value_per_acre(lat, long) * acres

def get_area_from_scan_track(scan, track):
    l = scan * 111 #km
    h = track * 111 #km
    area_in_km2 = l*h
    return area_in_km2

def get_weighted_economic_loss(total_economic_loss, risk, risk_scale): #risk_scale is maximum value of risk possible (such as out of 5 or out of 100 etc)
    if(risk > risk_scale):
        raise ArithmeticError("Risk cannot be greater than risk scale")
    return total_economic_loss * risk / risk_scale

In [12]:
# testing code
totalloss = get_total_economic_loss(32.9715645, -89.7348497, get_area_from_scan_track(3.5,1.8)) # roughly 0.00404686 km2 = 1 acre
weightedloss = get_weighted_economic_loss(totalloss, 3.7, 5)
print(totalloss, weightedloss)

1382939893632.1501 1023375521287.7913


# Location Name to Coordinates Tuple

In [13]:
def get_lat_long_from_location(name):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="myGeocoder")
    location = geolocator.geocode(name)
    return (location.latitude,location.longitude)

In [14]:
get_lat_long_from_location('Mississipi')

(32.9715645, -89.7348497)

# Ecological Loss Calculation

### Endangered Species

In [15]:
def get_county_name(lat, long):
    from geopy.geocoders import Nominatim
    locator = Nominatim(user_agent="myGeocoder")
    coordinates = str(lat)+", "+str(long)
    location = locator.reverse(coordinates)
    county = location.raw['address']["county"]
    if 'County' in county:
        county = county.replace(' County','')
    return county

In [16]:
def get_df_endangered_species(lat, long):
    county = get_county_name(lat, long)
    wild = pd.read_csv("Data/wildlife.csv")
    wild = wild.loc[(wild['County'] == county) & (wild['Global Conservation Rank'].isin(['G1','G2']))]['Common Name']
    if wild.empty:
        return None
    return wild

In [34]:
specs = get_df_endangered_species(42.6526, -73.7562) # if none return or no output - No endangered species

In [35]:
"<br>".join(specs.values)

'Rusty-patched Bumble Bee<br>Indiana Bat<br>Pitch Pine-Scrub Oak Barrens'

In [19]:
DATA_DIR = "../Data/FIRMS/"

FIIRMS_DF = pd.read_csv(DATA_DIR+"SUOMI_VIIRS_C2_USA_contiguous_and_Hawaii_7d.csv")

In [20]:
FIIRMS_DF

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,confidence,version,bright_ti5,frp,daynight
0,31.15063,-87.99097,316.7,0.49,0.65,2020-09-26,648,N,nominal,2.0NRT,289.4,2.4,N
1,31.15034,-87.98576,314.1,0.49,0.65,2020-09-26,648,N,nominal,2.0NRT,290.4,2.4,N
2,30.48426,-91.18553,300.2,0.74,0.76,2020-09-26,648,N,nominal,2.0NRT,289.5,2.1,N
3,30.34416,-88.48821,304.4,0.51,0.66,2020-09-26,648,N,nominal,2.0NRT,291.5,1.4,N
4,30.32692,-91.28209,299.0,0.75,0.77,2020-09-26,648,N,nominal,2.0NRT,288.4,1.1,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...
35678,47.57269,-122.37018,332.3,0.51,0.41,2020-10-02,2118,N,nominal,2.0NRT,297.3,4.4,D
35679,49.44254,-121.92496,326.6,0.56,0.43,2020-10-02,2118,N,nominal,2.0NRT,291.6,3.1,D
35680,49.44445,-121.92501,327.8,0.56,0.43,2020-10-02,2118,N,nominal,2.0NRT,292.1,3.1,D
35681,49.92025,-115.64212,367.0,0.35,0.57,2020-10-02,2118,N,high,2.0NRT,311.4,35.8,D


In [23]:
FIIRMS_DF['conf'] = FIIRMS_DF['confidence'].map({
    "low":0.2,
    "nominal":0.5,
    "high":0.9
})

In [32]:
get_ec(FIIRMS_DF.loc[1])

(102012330017.871, 51006165008.9355)

In [31]:
from tqdm import tqdm

tqdm.pandas()

def get_ec(row):
    loss = get_total_economic_loss(row['latitude'], row['longitude'], get_area_from_scan_track(row['scan'],row['track']))
    wt_loss = get_weighted_economic_loss(loss, row['conf'], 1)
    return loss, wt_loss
    

losses = FIIRMS_DF.progress_apply(get_ec, axis=1, result_type='expand')

  0%|          | 12/35683 [00:05<4:54:39,  2.02it/s]


TypeError: `address` must not be None